# **MICN: Multi-Scale Local and Global Context Modeling for Long-Term Series Forecasting**

Re-implementation of the [original paper](https://openreview.net/pdf?id=zt53IDUR1U)

Students:

- *Francesco Sudoso 1808353*
- *Francesco Sasanelli 2014433*


# **1. Description of the Paper**
In this project we reimplemented the original paper MICN: Multi-Scale Local and Global Context Modeling for Long-Term Series Forecasting.

The main goal of the authors is to address the challenges posed by Transformer-based methods in long-term series forecasting, particularly their high complexity in computing global correlations (caused by the attention mechanism) and the lack of targeted local feature modeling.

For this reason, they propose a new method called Multi-Scale Isometric Convolution Network (**MICN**) that combines local feature extraction with global correlation modeling, achieving greater accuracy and efficiency than existing methods. The approach is based on *convolutional operations*, demonstrating the advantages of this method over using *Transformers*.

The most important results of this implementation are the following:
- In order to achieve linear computational complexity, they have developed a convolution-based structure to replace the self-attention mechanism.
- Develop a local-global architecture for extraction and aggregation of informations and patterns, enabling effective long-term dependency modeling and outperforming self-attention mechanisms.

# **2. Main aspects of MICN**
We will see that MICN uses multiple branches, each obtained by different convolution kernels, to capture various potential patterns within the time series. Each branch extracts local features using a downsampling convolution, focusing on capturing short-term changes in the data. On top of the local feature extraction, MICN uses isometric convolution to model global correlations. After extracting local features and modeling global correlations, the information from various branches is merged.
The superiority of their local-global module is shown by comparing previous models like Autoformer, which uses auto-correlation.

# **3. Pytorch Code**

First of all, we import the dependencies that will be used to execute the code:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch.optim.lr_scheduler as lr_scheduler
import os
import numpy as np
import math
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader

## 3.1 Dataset and Dataloader implementation
Regarding the dataset, the dataloader and the related split between train, validation and test set, we followed what the author said:

> We follow standard protocol (Zhou et al., 2021) and split all datasets into training, validation and test set in chronological order by the ratio of 6:2:2 for the ETT dataset and 7:1:2 for the other datasets.

The protocol referred by the author is the one defined by the [Informer](https://arxiv.org/abs/2012.07436) paper (model that will be used as a comparison).

Our initial idea was to split any dataset into 80/10/10.
But realizing that there is a precise protocol for managing these specific datasets, we chose to follow their approach. Therefore, the approach used by Informer (and by all the models used in performance comparison) consists in splitting the "ETTm2" dataset based on the quarters of an hour present in a month (given that the measurements of electricity consumption of the dataset are taken every quarter of an hour). In particular, the train set contains 12 months of measurements, and the validation and test sets 4 and 4 respectively.
All other datasets will follow the 70/10/20 rule.

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, path, seq_len, pred_len, type, freq, device):
        self.type = type
        self.device = device
        self.seq_len, self.pred_len = seq_len, pred_len

        df = pd.read_csv(path)
        self.feature_dim = len(df.columns)-1

        if path == 'ETTm2.csv':
            QUARTER_HOURS_IN_ONE_MONTH = 30 * 24 * 4
            if type == 'train': #12 months
                start_row, end_row = 0, 12 * QUARTER_HOURS_IN_ONE_MONTH
            elif type == 'val': #4 months
                start_row, end_row = 12 * QUARTER_HOURS_IN_ONE_MONTH - self.seq_len, 16 * QUARTER_HOURS_IN_ONE_MONTH
            elif type == 'test': #4 months
                start_row, end_row = 16 * QUARTER_HOURS_IN_ONE_MONTH - self.seq_len, 20 * QUARTER_HOURS_IN_ONE_MONTH
        else:
            rows = len(df)
            if type == 'train':
                start_row, end_row = 0, int(rows * 0.7)
            elif type == 'val':
                start_row, end_row = int(rows * 0.7) - self.seq_len, int(rows * 0.8)
            elif type == 'test':
                start_row, end_row = int(rows * 0.8) - self.seq_len, rows

        #Feature Scaling
        scaler = StandardScaler()
        all_features = df[df.columns[1:]]
        train_features = all_features[0:12*QUARTER_HOURS_IN_ONE_MONTH] if path == 'ETTm2.csv' else all_features[0:int(len(df)*0.7)]
        scaler.fit(train_features.values)
        data = scaler.transform(all_features.values)

        self.time_features = self.get_time_features(df.iloc[start_row:end_row, 0], freq)
        self.data_x = data[start_row:end_row]
        self.data_y = data[start_row:end_row]

    def __len__(self):
        #Denotes the total number of samples
        return len(self.data_x) - self.seq_len - self.pred_len + 1

    def __getitem__(self, index):
        #Generates one sample of data
        seq_x = torch.from_numpy(self.data_x[index : (index+self.seq_len)]).float().to(self.device)
        seq_y = torch.from_numpy(self.data_y[index : (index+self.seq_len+self.pred_len)]).float()
        time_x = torch.from_numpy(self.time_features[index : (index+self.seq_len)]).float().to(self.device)
        time_y = torch.from_numpy(self.time_features[index : (index+self.seq_len+self.pred_len)]).float().to(self.device)
        return seq_x, seq_y, time_x, time_y

    def get_time_features(self, dates, freq):
        #Convert a timestamp to vector feature
        features = pd.DataFrame({'date': pd.to_datetime(dates)})
        features['month'] = features['date'].dt.month
        features['day'] = features['date'].dt.day
        features['weekday'] = features['date'].dt.weekday
        features['hour'] = features['date'].dt.hour
        features['minute'] = features['date'].dt.minute // 15

        freq_map = {
            'y':[],'m':['month'],'w':['month'],'d':['month','day','weekday'],
            'b':['month','day','weekday'],'h':['month','day','weekday','hour'],
            't':['month','day','weekday','hour','minute'],
        }

        return features[freq_map[freq]].values

    def get_Dataloader(self):
        if self.type == 'test':
            loader = DataLoader(self, batch_size=32, shuffle=False, drop_last=True)
        else: #train or val
            loader = DataLoader(self, batch_size=32, shuffle=True, drop_last=True)
        return loader

## **3.2 MICN Framework and Architecture**

The overall structure of MICN is shown in the figure below.
![Logo](https://github.com/Awenega/MICN/blob/main/images/MICN.png?raw=true)
As we can see, it is composed by four blocks:
- The **Multi-scale Hybrid Decomposition Block**, which is used to decompose the input series.
- The **Embedding Block** to add positional information but also to map time-related features to a higher dimension
- The **MIC Block** to predict seasonal information.
- The **Regression Block** to predict trend-cyclical information.

Since a time series can be seen as the sum of its trend and seasonality (and noise), the outputs of the MIC block and the Regression block are summed.

![Time series decomp](https://github.com/Awenega/MICN/blob/main/images/decomp.PNG?raw=true)

### **3.2.1 MICN code**
<a class="anchor" name="3.2.1"></a>
To explain how we implemented the model, we will use a Bottom-Up approach: we will start with the code of each block previously listed, and then we go up to the model itself. Accordingly, we will provide the explanation of the following blocks in order:
1. MHD Block
2. Regression Block
3. Embedding Block
4. MIC Block (which is divided into Local and Global Blocks)
5. MICN model

In the next section we start the explaination from the MHD Block.

### **3.2.2 Multi-Scale Hybrid Decomposition Block**
One of the main methods of decomposing time series into their respective trends and seasonality is to use the moving average. In fact, this approach was also used to implement [Autoformer](https://arxiv.org/abs/2106.13008) and [FEDformer](https://arxiv.org/abs/2201.12740), which are models for predicting long-term series, which will be used as a comparison of the results that MICN will obtain.

This method consists of using Avgpool layer with fixed kernels. But the authors propose to use several different kernels for the AvgPool operation. This approach allows for the separation of various patterns in the trend-cyclical and seasonal parts of the series, that, at the end, are integrated through a mean operation.

The class *moving_avg* is taken from the [Autoformer](https://github.com/thuml/Autoformer/blob/main/layers/Autoformer_EncDec.py) implementation and we have done some modifications to make the code more readable and interpretable.

Instead, the class *Series_Decomposer* is built on top of the previous class. Specifically, depending on the case in which we need to decompose the series (whether with multiple kernels or whether with a single one), the class will consist of an array of avgpools, each with a different kernel. The purpose is to extract different patterns and eventually average them out.

This block produces two outputs: the first, which is the extracted seasonality, will serve as input to the next block we will discuss, which is the embedding. While the second, the extracted trend, will be used in the forward pass of the MICN model to apply regression or mean to make a prediction about the final trend.

In the next section, we will explain the Embedding block.

In [ ]:
class moving_avg(nn.Module):
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.pad_size = (kernel_size - 1) // 2
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front_padding = x[:, 0:1, :].repeat(1, self.pad_size, 1)
        end_padding = x[:, -1:, :].repeat(1, self.pad_size, 1)
        x_padded = torch.cat([front_padding, x, end_padding], dim=1)
        x_smoothed = self.avg(x_padded.permute(0, 2, 1))
        x_smoothed = x_smoothed.permute(0, 2, 1)
        return x_smoothed

class Series_Decomposer(nn.Module):
    def __init__(self, kernel_size, multi=False):
        super(Series_Decomposer, self).__init__()
        self.multi = multi
        self.moving_avg_layers = nn.ModuleList([moving_avg(kernel, stride=1) for kernel in kernel_size]) if multi else moving_avg(kernel_size, stride=1)

    def compute_moving_avgs(self, x):
        moving_means = []
        seasonals = []
        if self.multi:
            for layer in self.moving_avg_layers:
                moving_avg = layer(x)
                seasonal = x - moving_avg
                moving_means.append(moving_avg)
                seasonals.append(seasonal)
        else:
            moving_avg = self.moving_avg_layers(x)
            seasonal = x - moving_avg
            moving_means.append(moving_avg)
            seasonals.append(seasonal)

        return seasonals, moving_means

    def forward(self, x):
        seasonals, moving_means = self.compute_moving_avgs(x)
        seasonal = sum(seasonals) / len(seasonals)
        moving_mean = sum(moving_means) / len(moving_means)

        return seasonal, moving_mean

### **3.2.3 Embedding Block**

The Embedding Block is described in the Figure below.

![Embedding Block](https://github.com/Awenega/MICN/blob/main/images/Embedding.png?raw=true)

As we can see:
- $X_s$ is the output of the MHD block, so it is the seasonal decomposed part of the time series.
- $X_{zero}$ is a tensor of all zeros.
- $VE$, $PE$ and $TFE$ are respectively the Value Embedding, Positional Encoding and Time Feature Encoding.

$X_s$ is concatenated to $X_{zero}$ and then passed as input to *ValueEmbedding*. At the end, the output of *ValueEmbedding* is summed to the output of the *PositionalEncoding* and to the output of the *TimeFeatureEncoding*.

So, the first step is to concatenate $X_s$ and $X_{zero}$. We have done this step in the forward pass of the Embedding block. We report the code(not executable) that will be placed in the section of the MICN

```python
# embedding
zeros = torch.zeros([seq_x.shape[0], self.pred_len, seq_x.shape[2]], device=seq_x.device)
seasonal = seasonal[:, -self.seq_len:, :]
seasonal_padded_zeros = torch.cat([seasonal, zeros], dim=1)
```
Then, the next step is to proceed with the embedding itself. But, since the authors said:

> We follow the setting of FEDformer and adopt three parts to embed the input.<br>
The process is: $X_{emb}^s$ = sum(TFE + PE + VE(Concat($X_s$,$X_{zero}$)))

We decided to look at [FEDformer implementation](https://github.com/MAZiqing/FEDformer/blob/master/layers/Embed.py) and study their approach. But, looking deeper, we noticed that most models that predict long-term series use the same embedding technique. Consequently, it seemed the obvious choice for us to take this library as well and use it with our model. Of course, we modified some parts of the code to fit our model, but the basic operations are the same.





In [ ]:
# add positional information to the model
class PositionalEmbedding(nn.Module):
    def __init__(self, embedding_dim, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        pe = torch.zeros(max_len, embedding_dim).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, embedding_dim, 2).float() * -(math.log(10000.0) / embedding_dim)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]

# Utilizes a 1D convolutional layer to generate token embeddings from input sequences.
# Each token is represented by a dense vector of output_feature_dim dimensions.
class TokenEmbedding(nn.Module):
    def __init__(self, input_feature_dim, output_feature_dim):
        super(TokenEmbedding, self).__init__()
        self.tokenConv = nn.Conv1d(in_channels=input_feature_dim, out_channels=output_feature_dim,
                                    kernel_size=3, padding=1, padding_mode='circular')
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight,mode='fan_in',nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1,2)
        return x

# It maps time-related features to a higher-dimensional space
class TimeFeatureEmbedding(nn.Module):
    def __init__(self, output_dim, freq='t'):
        super(TimeFeatureEmbedding, self).__init__()

        freq_map = {'h':4, 't':5, 's':6, 'm':1, 'a':1, 'w':2, 'd':3, 'b':3}
        input_dim = freq_map[freq]
        self.embed = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.embed(x)

So, the final Embedding Block is composed by these three embedding, that in the forward pass, we sum them up:

In [ ]:
# Embedding Block
class DataEmbedding(nn.Module):
    def __init__(self, input_feature_dim, embedding_dim, freq, dropout, seq_len, pred_len):
        super(DataEmbedding, self).__init__()

        self.seq_len, self.pred_len = seq_len, pred_len

        self.value_embedding = TokenEmbedding(input_feature_dim, embedding_dim)
        self.position_embedding = PositionalEmbedding(embedding_dim=embedding_dim)
        self.temporal_embedding = TimeFeatureEmbedding(output_dim=embedding_dim, freq=freq)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, seq_x, seasonal, time):

        zeros = torch.zeros([seq_x.shape[0], self.pred_len, seq_x.shape[2]], device=seq_x.device)
        seasonal = seasonal[:, -self.seq_len:, :]
        seasonal_padded_zeros = torch.cat([seasonal, zeros], dim=1)

        seasonal_embedding = self.value_embedding(seasonal_padded_zeros) + self.position_embedding(seasonal_padded_zeros) + self.temporal_embedding(time)
        return self.dropout(seasonal_embedding)

### **3.2.4 MIC Block**
<a class="anchor" name="3.2.4"></a>

The MIC Block is described in the figure below.
![MIC Block](https://github.com/Awenega/MICN/blob/main/images/MIC.png?raw=true)

Here, we can see that the output of the Embedding Block is passed as input to the MIC block. It contains several branches, with different scale sizes used to model potentially different temporal patterns.

Moreover, the MIC block itself can be seen as the composition of 3 smaller blocks:
- The Local Block, that extracts the local features.
- The Global Block, that is designed to model the global correlations of the output of
the Local Block
- The Merge Block, that takes all the patterns and merge them together with Conv2d layer.

At the end, some other operations are performed to obtain the final result.

We will describe the code in order of the previously listed blocks.


#### **3.2.4.1 Local and Global Block**
<a class="anchor" name="3.2.4.1"></a>

From the figure below, we can see what layers the Local Block and the Global Block are composed of.

![Local Global Block](https://github.com/Awenega/MICN/blob/main/images/LocalGlobal.png?raw=true)


This is the Local Block. It is composed by many convolutional layers, one for each kernel we are using to decompose the series.

> For Conv1d, we set stride = kernel = i, which serves as compression of local features.



In [ ]:
class Local(nn.Module):
    def __init__(self, input_feature_dim, conv_kern, dropout):
        super(Local, self).__init__()
        self.conv = nn.ModuleList([nn.Conv1d(in_channels=input_feature_dim, out_channels=input_feature_dim,
                                             kernel_size=i,padding=i//2,stride=i)
                                  for i in conv_kern])
        self.tanh = torch.nn.Tanh()
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, seasonal, i):
        # Downsampling and then Tanh & Dropout
        seasonal_reshaped = seasonal.permute(0, 2, 1)
        return self.dropout(self.tanh(self.conv[i](seasonal_reshaped)))

Then, the output of this block is passed to the Global Block.<br>
This block is the one of the most important block in the paper since introduce a new approach to model the global correlations. Indeed:

> A commonly used method for modeling global correlations is the self-attention
mechanism. But in this paper, we use a variant of casual convolution, isometric convolution, as an alternative.

Before we can apply the isometric convolution, we should first:

> pads the sequence of length S with placeholders zero of length $S−1$

Since, in the Local Block we have downsapled the input, after the isometric convolution we upsample the sequence with ConvTranspose1d layer, as mentioned in the paper.



In [ ]:
class Global(nn.Module):
    def __init__(self, input_feature_dim, iso_kern, conv_kern, dropout):
        super(Global, self).__init__()

        self.isometric_conv = nn.ModuleList([nn.Conv1d(in_channels=input_feature_dim, out_channels=input_feature_dim,
                                                   kernel_size=i,padding=0,stride=1)
                                        for i in iso_kern])
        self.tanh = torch.nn.Tanh()
        self.dropout = torch.nn.Dropout(dropout)
        self.norm = torch.nn.LayerNorm(input_feature_dim)

        self.upsample_conv = nn.ModuleList([nn.ConvTranspose1d(in_channels=input_feature_dim, out_channels=input_feature_dim,
                                                            kernel_size=i,padding=0,stride=i)
                                        for i in conv_kern])

    def forward(self, seasonal, local_out, i):
        # we first pad the sequence
        zeros = torch.zeros((local_out.shape[0], local_out.shape[1], local_out.shape[2]-1), device=local_out.device)
        local_out_padded = torch.cat((zeros, local_out), dim=-1)
        # then we apply isometric convolution
        ret = self.dropout(self.tanh(self.isometric_conv[i](local_out_padded)))

        iso_conv_result = local_out + ret
        iso_conv_result = self.norm(iso_conv_result.permute(0, 2, 1)).permute(0, 2, 1)

        # upsampling convolution
        upsample_conv = self.dropout(self.tanh(self.upsample_conv[i](iso_conv_result)))

        # Minimal truncation (the impact on the overall information content should be relatively small)
        # It is done for computing (seasonal + upsample_conv), since both must have same shape
        upsample_conv = upsample_conv[:, :, :seasonal.shape[1]].permute(0, 2, 1)

        return self.norm(seasonal + upsample_conv)

Then, we can put Local Block and Global Block together for composing the Local_Global module illustrated in the previous [figure](#3.2.4.1).

The first operation is to decompose with different kernels the seasonal component in order to extract all possible informations. Each decomposition, follows the Local Block that will extract the local feature and the Global block to model the global correlation. At the end of the process, we will have all the patterns that should be merged in the upcoming operations.


In [ ]:
class Local_Global(nn.Module):
    def __init__(self, input_feature_dim, decomp_kern, conv_kern, iso_kern, dropout):
        super(Local_Global, self).__init__()

        self.decomp = nn.ModuleList([Series_Decomposer(kernel, multi=False) for kernel in decomp_kern])
        self.local_block = Local(input_feature_dim, conv_kern, dropout)
        self.global_block = Global(input_feature_dim, iso_kern, conv_kern, dropout)

    def forward(self, seasonal_embedded, conv_kern):
        patterns = []
        for i in range(len(conv_kern)):
            seasonal, _ = self.decomp[i](seasonal_embedded)
            local_out = self.local_block(seasonal, i)
            global_out = self.global_block(seasonal, local_out, i)
            patterns.append(global_out)
        return patterns

At this point we have finished to implement the core part of [MIC](#3.2.4). The only things that are missing are the merge operation, the FeedForward and few other small operations.

We first show the code for the FeedForward. It is a very simple class that can be implemented in many ways, but we preferred to make it as small and simple as possible to avoid increasing too much the number of parameters and computational complexity that we will have to deal with later during training.

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, input_size, hidden_size, dropout):
        super(FeedForward, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, input_size)
        )

    def forward(self, patterns_merged):
        return self.net(patterns_merged)

Now, we can build the main block of the model, which is the MIC Block. It follows exactly the formulas described in the paper:

 - $Y_{s,l}^{merge} = (Conv2d(Y_{s,l}^{global,i}) $ <br><br> Here, we are merging all the results obtained from the Local_Global Block.<br><br>
 - $ Y_{s,l} = Norm(Y_{s,l}^{merge} + FeedForward(Y_{s,l}^{merge})) $ <br><br>
 Here, we are passing all the patterns merged to the feedforward network and then normalizing the sum of the patterns merged itself and the output. <br><br>
 - $ Y_s = Truncate(Projection(Y_{s,N})) $ <br><br>
 Here, we compute the final output of the prediction of the seasonal component by the projection and truncate operation on the previous output. <br><br>
 $ Y_S $ represents the final prediction about the seasonal part.



In [ ]:
class MIC(nn.Module):
    def __init__(self, input_feature_dim, output_feature_dim, decomp_kern, conv_kern, iso_kern, dropout):
        super(MIC, self).__init__()
        self.conv_kern = conv_kern

        self.local_global = Local_Global(input_feature_dim, decomp_kern, conv_kern, iso_kern, dropout)
        self.merge = torch.nn.Conv2d(in_channels=input_feature_dim, out_channels=input_feature_dim, kernel_size=(len(conv_kern), 1))

        self.feedForward = FeedForward(input_feature_dim, input_feature_dim*4, dropout)

        self.norm = torch.nn.LayerNorm(input_feature_dim)
        self.projection = nn.Linear(input_feature_dim, output_feature_dim)

    def forward(self, seasonal_embedded):
        patterns = self.local_global(seasonal_embedded, self.conv_kern)
        patterns_list = [pattern.unsqueeze(1) for pattern in patterns]
        # Merge operation is done by Conv2d layer. We need to change the shape of the input tensor
        patterns_tensor = torch.cat(patterns_list, dim=1).permute(0, 3, 1, 2).to('cuda:0')
        patterns_merged = self.merge(patterns_tensor).squeeze(-2).permute(0,2,1)
        # Merge operation finished, then we remove the dimension that we added before with the unsqueeze(1) and we permute the shape back to the original shape

        ret = self.norm(patterns_merged + self.feedForward(patterns_merged))
        ret = self.projection(ret)
        return ret

### **3.2.5 MICN model**

Now, we can pass to the MICN model itself. As mentioned earlier, the model consists of 4 main blocks.

The first block that is used and applied in the forward is the MHD block, which is used to decompose the series into trend and seasonality.

After that, the model uses a method of our choice between regression and mean to make a prediction about trend-cyclical.

Next, padding is applied to the previously extracted seasonality to be serially passed as input to the embedding block and MIC block.

The output of the MIC block will contain the prediction of the seasonality while the output of the regression or mean block, will contain the prediction of the trend.
These two outputs will be summed to provide the final prediction.


In [ ]:
class MICN(nn.Module):
    def __init__(self, input_feature_dim, output_feature_dim, seq_len, pred_len, embedding_dim, dropout, freq, mode,
                decomp_kern, conv_kern, iso_kern):
        super(MICN, self).__init__()

        self.seq_len, self.pred_len, self.mode = seq_len, pred_len, mode

        #MHD Block
        self.decomp = Series_Decomposer(decomp_kern, multi=True)
        #Embedding Block
        self.embedding = DataEmbedding(input_feature_dim=input_feature_dim, embedding_dim=embedding_dim, freq=freq, dropout=dropout, seq_len = seq_len, pred_len = pred_len)
        #MIC Block
        self.mic = MIC(input_feature_dim=embedding_dim, output_feature_dim=output_feature_dim, decomp_kern=decomp_kern, conv_kern=conv_kern, iso_kern=iso_kern, dropout=dropout)
        #Regression Block
        self.regression = nn.Linear(seq_len, pred_len)

    def forward(self, seq_x, time_y):

        #series decomposition
        seasonal, trend = self.decomp(seq_x)

        #trend prediction
        if self.mode == 'regression':
            trend_pred = self.regression(trend.permute(0,2,1)).permute(0, 2, 1)
        elif self.mode == 'mean':
            mean = torch.mean(seq_x, dim=1).unsqueeze(1).repeat(1, self.pred_len, 1)
            trend_pred = torch.cat([trend[:, -self.seq_len:, :], mean], dim=1)

        #seasonal embedding
        seasonal_embed = self.embedding(seq_x, seasonal, time_y)
        #seasonal prediction
        seasonal_pred = self.mic(seasonal_embed)
        #timeseries prediction
        time_series_pred = seasonal_pred[:, -self.pred_len:, :] + trend_pred[:, -self.pred_len:, :]

        return time_series_pred

## 3.3 **Training code**

In the implementation details of the paper it is mentioned that:
> The training process is early stopped after three epochs if there is no loss degradation on the valid set.

For this reason, we implemented a class to handle the early stopping.
We took an already implemented and predefined [class](https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py
) and adapted it to our case.

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, logger=None):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), path+'/'+'checkpoint.pth')
        self.val_loss_min = val_loss

Then, we set some hyperparameters and some variable that we will use for the train. In this case, is possible to change the variable 'mode'.<br>
After, is possible to choose the dataset that we want to have as input by changing the 'filename_dataset' variable.<br> The variables 'freq', 'seq_len', 'pred_len' and the kernels will depend on the dataset choosen: <br>
- Regarding the 'freq', it is the sampling time for each dataset. For example, the dataset "ETTm2" contains electric consumption measurements taken every 15 minutes from July 2016 to July 2018. The table below define all the info about this sampling. The variable will be used for the vector decomposition of the date of the measurement (during the embedding).
- For the prediction lenght, we can choose between 24,36 and 48 for national_illness dataset. For the other datasers, we can choose between 96, 192 and 336.
- Regarding the kernels, we have set the kernals that the authors have used for their tests.

Also, we create the dataloaders from the dataset.

![](https://github.com/Awenega/MICN/blob/main/images/datasets.png?raw=true)

In [ ]:
#Setup of hyper-parameters
mode = 'regression' # 'regression' or 'mean'
embedding_dim = 512
dropout = 0.05
lr = 0.001
gamma = 0.5
patience = 3
epochs = 20
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
os.makedirs('./checkpoints/') if not os.path.exists('./checkpoints/') else None

#Setup of dataset filename and its respective frequency of data sampling
filename_dataset = 'electricity.csv'
if filename_dataset == 'ETTm2.csv' or filename_dataset == 'weather.csv':
    freq = 't'
elif filename_dataset == 'traffic.csv' or filename_dataset == 'electricity.csv':
    freq = 'h'
else:
    freq = 'd'

#Setup of seq_len and kernels depending on the dataset
if filename_dataset != 'national_illness.csv':
    seq_len = 96
    pred_len = 96 # 96 or 192 or 336
    conv_kern, decomp_kern = [12, 16], [13, 17]
else:
    seq_len = 36
    pred_len = 24 # 24 or 36 or 48
    conv_kern, decomp_kern = [12, 16], [13, 17]
iso_kern = [(seq_len + pred_len + kern) // kern for kern in conv_kern]

#Setup of the dataloaders
train_data, val_data, test_data = [Dataset(f'{filename_dataset}', seq_len, pred_len, type, freq, device) for type in ['train', 'val', 'test']]
train_loader, val_loader, test_loader = [dataset.get_Dataloader() for dataset in [train_data, val_data, test_data]]
feature_dim = train_data.feature_dim

#Setup of the model
model = MICN(input_feature_dim = feature_dim, output_feature_dim = feature_dim, seq_len = seq_len, pred_len = pred_len, embedding_dim=embedding_dim, freq=freq, mode=mode,
                dropout=dropout, conv_kern=conv_kern, decomp_kern=decomp_kern,  iso_kern=iso_kern).float().to(device=device)

#Setup of the optimizer, the scheduler for adjusting the learning rate during the training, and the early stopping
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
criterion = nn.MSELoss()
early_stopping = EarlyStopping(patience=patience, verbose=True)

Then we have the training code, followed by the validation process.

In [ ]:
print(f"dataset: {filename_dataset}, feature_dim: {feature_dim}, pred_len: {pred_len}, iso_kern: {iso_kern}, mode: {mode}")
for epoch in range(epochs):

            # Training Loop
            train_losses = []
            model.train()
            for batch, (seq_x, seq_y, time_x, time_y) in enumerate(train_loader, 1):
                # called at the beginning of the training step for each batch to ensure that the gradients computed for the current batch are not influenced by gradients from the previous batches.
                optimizer.zero_grad()

                #forward pass compute predicted outputs
                output = model(seq_x, time_y)
                y_train = seq_y[:,-pred_len:,:].to(device)

                # calculate the loss
                loss = criterion(output, y_train)
                #backward pass: computes the gradients of the loss with respect to the model parameters
                loss.backward()
                #once the gradients are computed, model.step() update the parameters
                optimizer.step()
                if (batch+1) % 100 == 0:
                    print(f"iter: {batch+1} | epoch: {epoch+1} | loss: {loss.item():.5f}")
                train_losses.append(loss.item())

            # Validation Loop
            model.eval()
            with torch.no_grad():
                val_losses = []
                for i, (seq_x, seq_y, time_x, time_y) in enumerate(val_loader):

                    #forward pass compute predicted outputs
                    output = model(seq_x, time_y)
                    y_val = seq_y[:,-pred_len:,:].to(device)

                    loss = criterion(output, y_val)
                    val_losses.append(loss.item())

            # calculate average loss over an epoch
            train_loss = np.average(train_losses)
            val_loss = np.average(val_losses)
            print(f"\nEpoch: {epoch+1} | Train Loss: {train_loss:.5f} | Vali Loss: {val_loss:.5f}")

            early_stopping(val_loss, model, './checkpoints/')
            if early_stopping.early_stop:
                break

            print(f"Updating learning rate to {scheduler.get_last_lr()}")
            scheduler.step()

dataset: electricity.csv, feature_dim: 321, pred_len: 96, iso_kern: [17, 13], mode: regression


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


iter: 100 | epoch: 1 | loss: 0.22630
iter: 200 | epoch: 1 | loss: 0.21461
iter: 300 | epoch: 1 | loss: 0.19104
iter: 400 | epoch: 1 | loss: 0.15732
iter: 500 | epoch: 1 | loss: 0.16960

Epoch: 1 | Train Loss: 0.21424 | Vali Loss: 0.14211
Validation loss decreased (inf --> 0.142110).  Saving model ...
Updating learning rate to [0.001]
iter: 100 | epoch: 2 | loss: 0.14477
iter: 200 | epoch: 2 | loss: 0.15355
iter: 300 | epoch: 2 | loss: 0.17048
iter: 400 | epoch: 2 | loss: 0.13833
iter: 500 | epoch: 2 | loss: 0.13493

Epoch: 2 | Train Loss: 0.14223 | Vali Loss: 0.13075
Validation loss decreased (0.142110 --> 0.130753).  Saving model ...
Updating learning rate to [0.0005]
iter: 100 | epoch: 3 | loss: 0.11894
iter: 200 | epoch: 3 | loss: 0.13059
iter: 300 | epoch: 3 | loss: 0.11882
iter: 400 | epoch: 3 | loss: 0.12057
iter: 500 | epoch: 3 | loss: 0.11668

Epoch: 3 | Train Loss: 0.12526 | Vali Loss: 0.12810
Validation loss decreased (0.130753 --> 0.128102).  Saving model ...
Updating learni

Finally, we have the evaluation process that will compute the performance of the model (MSE and MAE).

In [ ]:
# Evaluation
print('Evaluation:')
model.load_state_dict(torch.load('./checkpoints/checkpoint.pth'))
model.eval()

predictions, truths = [], []
with torch.no_grad():
    for _, (seq_x, seq_y, time_x, time_y) in enumerate(test_loader):

        #forward pass compute predicted outputs
        prediction = model(seq_x, time_y)
        truth = seq_y[:,-pred_len:,:].to(device)

        predictions.append(prediction)
        truths.append(truth)

#Convert lists into pytorch tensors
predictions_tensor, truths_tensor = torch.stack(predictions), torch.stack(truths)

# Compute Mean Squared Error (MSE) and Mean Absolute Error (MAE)
mse = torch.mean((predictions_tensor - truths_tensor) ** 2).item()
mae = torch.mean(torch.abs(predictions_tensor - truths_tensor)).item()
print(f'MSE: {mse:.5f} | MAE: {mae:.5f}')

Evaluation:
MSE: 0.16290 | MAE: 0.27200


# **4. Results**

In the following, we will show the results obtained from the paper author's model and compare them with the results obtained from our reimplemented model. Specifically, for each of the datasets we performed 3 tests, since there is randomness during the training process (e.g. due to the shuffling applied by the dataloader or from the initialization to random values of some weights). The numbers we have included in the table below, derived from the average of the values obtained from the 3 tests.

Some tests were performed locally through a NVIDIA RTX 3050 Ti 4GB GPU. Other tests, due to the limited memory of the video card, were performed on Google Colab with an NVIDIA T4 GPU provided by Colab.

Some types of tests, i.e., those involving datasets with very large and with more than 800 features or with a prediction length of 720, we were unable to perform the training because of lack of computational power of the GPU, or because of the time it would have been necessary to keep the Colab session active.We marked them in the table with '---' signature.

As we can see from the table below, we were able to obtain results very close to those presented in the paper. The table on the left represents the results of the original paper, while the table on the right represents the results of our model.

![Results](https://github.com/Awenega/MICN/blob/main/images/results.png?raw=true)

The following table, instead, represents the comparison of the model proposed by the original paper with all other models that address the same problem, that is, long-term series forecasting.

Bold results are the best ones. As we can see, MICN completely outperforms all other models.

![Results comparison](https://github.com/Awenega/MICN/blob/main/images/original%20results.png?raw=true)